# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
mongoimport --db posts --collection rows --file establishments.json --jsonArray

SyntaxError: invalid syntax (3838815017.py, line 1)

AttributeError: 'str' object has no attribute 'establishment'

Available databases:
admin
config
epa
fruits_db
local


'uk_food' database is not present.


In [10]:
# Check if 'establishments' collection is in the list


'establishments' collection is not present.


None


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [18]:
from pymongo import MongoClient

# Connect to the MongoDB client
mongo = MongoClient(port=27017)

# Access the database and collection
db = mongo['your_database_name']  # Replace 'your_database_name' with the actual database name
establishments = db['establishments']

# Find establishments with a hygiene score equal to 20
establishments_with_score_20 =establishments.find({"scores.Hygiene": 20})

# Print the results
for establishment in establishments_with_score_20:
    print(establishment)

In [19]:
# Use count_documents to display the number of documents in the result
count = establishments_with_score_20.count_documents()


AttributeError: 'Cursor' object has no attribute 'count_documents'

In [20]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments_with_score_20))

In [21]:
# Display the number of rows in the DataFrame
num_rows = df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 0


In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments_with_score_20))
# Display the number of rows in the DataFrame
num_rows = df.shape[0]
print("Number of rows in the DataFrame:", num_rows)
# Display the first 10 rows of the DataFrame


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [24]:
# Define the query
query = {
    "LocalAuthorityName": "London",
    "scores.RatingValue": {"$gte": 4}
}

# Find establishments matching the query
result = establishments.find(query)

# Iterate over the results and print each document
for doc in result:
    pprint(doc)

In [25]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

In [26]:
# Display the number of rows in the DataFrame
num_rows = df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 0


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [27]:
from bson.son import SON

# Get the coordinates of "Penang Flavours"
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
penang_coordinates = [penang_flavours["geocode"]["longitude"], penang_flavours["geocode"]["latitude"]]

# Define the query
query = {
    "scores.RatingValue": 5,
    "geocode": {"$exists": True, "$ne": None}
}

# Define the sort criteria
sort_criteria = [
    ("scores.Hygiene", 1),  # Sort by hygiene score in ascending order
    ("geocode", SON([("$near", penang_coordinates)]))  # Sort by distance from "Penang Flavours"
]

# Find the establishments matching the query and sort them
result = establishments.find(query).sort(sort_criteria).limit(5)

# Print the top 5 establishments
for doc in result:
    pprint(doc)

TypeError: 'NoneType' object is not subscriptable

In [28]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(result))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
pipeline = [
    {"$match": {"scores.Hygiene": "0"}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

result = establishments.aggregate(pipeline)

for doc in result:
    print(doc)

In [ ]:

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

In [ ]:

# Display the number of rows in the DataFrame
num_rows = df.shape[0]
print("Number of rows in the DataFrame:", num_rows)